In [1]:
import numpy as np
import pandas as pd

import edhec_risk_kit_206 as erk

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
ind_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2014":"2018"]
ind_mcap = erk.get_ind_market_caps(49, weights=True)["2014":"2018"]
cw = erk.weight_cw(ind_rets, ind_mcap)

In [5]:
def risk_contribution(w,cov):
    """
    Compute the contributions to risk of the constituents of a portfolio, given a set of portfolio weights and a covariance matrix
    """
    total_portfolio_var = erk.portfolio_vol(w,cov)**2
    # Marginal contribution of each constituent
    marginal_contrib = cov@w
    risk_contrib = np.multiply(marginal_contrib,w.T)/total_portfolio_var
    return risk_contrib

In [8]:
cov = ind_rets.cov()
risk_contribution(cw, cov).sort_values(ascending=False).head()

Banks    0.103966
Softw    0.098402
Drugs    0.089100
Oil      0.085238
Rtail    0.067192
dtype: float64

In [38]:
ew = erk.weight_ew(ind_rets)
risk_contribution(ew, cov).sort_values(ascending=False).head()

Steel    0.030938
Ships    0.030005
Coal     0.029025
Mines    0.028641
BldMt    0.028106
dtype: float64

In [12]:
def weight_erc(r, cov_estimator=erk.sample_cov, **kwargs):
    """
    Produces the weights of the ERC portfolio given a covariance matrix of the returns 
    """
    est_cov = cov_estimator(r, **kwargs)
    return equal_risk_contributions(est_cov)

In [28]:
werc = pd.Series(weight_erc(ind_rets, cov_estimator=erk.sample_cov), index=ind_rets.columns)

In [29]:
werc.sort_values(ascending=False).head()

Util     0.052156
Hshld    0.033184
Smoke    0.032606
Beer     0.032028
Food     0.030251
dtype: float64

In [30]:
werc.sort_values().head()

Steel    0.012809
Ships    0.013041
BldMt    0.013790
Mines    0.013951
Mach     0.014292
dtype: float64

In [37]:
risk_contribution(cw, cov).sort_values().iloc[[0,-1]].diff()

Gold          NaN
Banks    0.103872
dtype: float64

In [39]:
risk_contribution(ew, cov).sort_values().iloc[[0,-1]].diff()

Util          NaN
Steel    0.025022
dtype: float64